# Pruebas MultiFADL One-versus-Rest. water dataset

In [14]:
import pickle
import json

import pandas as pd
import tensorflow as tf
import numpy as np
from src import utils

from src.MonoFADLModel import MonoFADLModel
from src.MultiFADLModelOvR import MultiFADLModelOvR
from src.NoSelectionModel import NoSelectionModel

# Seed for neural network executions
SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [15]:
comparative_results = pd.DataFrame(columns=['test_loss', 'test_accuracy', 'test_f1', 'number of selected features', 'selected Features'])

* Preprocesamiento

In [16]:
# Load dataset and preprocess it

# Check if stored, otherwise load and store
try:
    water = pickle.load(open('data/water_pump.pkl', 'rb'))
except:
    water = pd.read_csv('data/water_pump.csv')
    
    # Drop unuseful columns
    water = water.drop('recorded_by_GeoData Consultants Ltd', axis=1) 
    
    # Target codification as label encoding, eliminate OneHot target
    target_mapping = {
    (0, 0, 1): 0,  # No funcional
    (0, 1, 0): 1,  # Funcional necesitando reparación
    (1, 0, 0): 2,  # Funcional
    }
    water['target'] = water.apply(lambda row: 
                                          target_mapping[(row['target_functional'], 
                                                          row['target_functional needs repair'], 
                                                          row['target_non functional'])], 
                                          axis=1)
    water = water.drop(['target_functional', 'target_functional needs repair', 'target_non functional'], axis=1)


    with open('data/water_pump.pkl', 'wb') as f:
        pickle.dump(water, f, protocol=4)

water

,amount_tsh,gps_height,longitude,latitude,num_private,population,funder_0,funder_A/co Germany,funder_Aar,funder_Abas Ka,...,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other,target
0,6000.0,1390,34.938093,-9.856322,0,109,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
1,0.0,1399,34.698766,-2.147466,0,280,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
2,25.0,686,37.460664,-3.821329,0,250,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
3,0.0,263,38.486161,-11.155298,0,58,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.0,0,31.130847,-1.825359,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,1210,37.169807,-3.253847,0,125,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
59396,4700.0,1212,35.249991,-9.070629,0,56,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
59397,0.0,0,34.017087,-8.750434,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,2
59398,0.0,0,35.861315,-6.378573,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,2


In [17]:
# Stratify train-val-test split
X_train, X_val, X_test, y_train, y_val, y_test = utils.train_test_split_stratify(df=water, target='target', SEED=1223)

# Standardization of continuous variables
X_train_scaled, X_val_scaled, X_test_scaled = utils.scale_numerical_variables(X_train, X_test, X_val,
                                                                              numerical_variables = ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'population']
                                                                             )

Class distribution in train:
2    19355
0    13694
1     2591
Name: target, dtype: int64
Class distribution in val:
2    6452
0    4565
1     863
Name: target, dtype: int64
Class distribution in test:
2    6452
0    4565
1     863
Name: target, dtype: int64


In [18]:
X_train_scaled

,amount_tsh,gps_height,longitude,latitude,num_private,population,funder_0,funder_A/co Germany,funder_Aar,funder_Abas Ka,...,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
48806,0.000362,0.005295,0.994599,0.121630,0.0,0.040,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
20019,0.000000,0.022238,0.837546,0.182041,0.0,0.000,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
45849,0.007246,0.022238,0.962298,0.584830,0.0,0.005,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
44776,0.003623,0.482880,0.941852,0.629626,0.0,0.012,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
54872,0.014493,0.566890,0.939306,0.634273,0.0,0.042,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2858,0.000000,0.815037,0.857573,0.179814,0.0,0.002,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6194,0.000000,0.022238,0.839355,0.255833,0.0,0.000,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
22360,0.000000,0.022238,0.759150,0.760444,0.0,0.000,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
59232,0.000000,0.022238,0.834370,0.227786,0.0,0.000,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


* Noselection results

In [19]:
model1 = NoSelectionModel(
    n_inputs=X_train_scaled.columns.values.shape[0],
    n_class=y_train.unique().shape[0]
)       

model1.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

model1.model.save('results/water_pump/water_pump_NoSelectionModel.h5')
with open('results/water_pump/water_pump_NoSelectionModel_history.pkl', 'wb') as f:
    pickle.dump(model1.history, f)

Epoch 1/50
1114/1114 [==============================] - 4s 3ms/step - loss: 0.5938 - sparse_categorical_accuracy: 0.7561 - val_loss: 0.5320 - val_sparse_categorical_accuracy: 0.7810
Epoch 2/50
1114/1114 [==============================] - 2s 2ms/step - loss: 0.4930 - sparse_categorical_accuracy: 0.7954 - val_loss: 0.5177 - val_sparse_categorical_accuracy: 0.7869
Epoch 3/50
1114/1114 [==============================] - 2s 2ms/step - loss: 0.4471 - sparse_categorical_accuracy: 0.8126 - val_loss: 0.5214 - val_sparse_categorical_accuracy: 0.7906
Epoch 4/50
1114/1114 [==============================] - 2s 2ms/step - loss: 0.4153 - sparse_categorical_accuracy: 0.8247 - val_loss: 0.5319 - val_sparse_categorical_accuracy: 0.7892
Epoch 5/50
1114/1114 [==============================] - 2s 2ms/step - loss: 0.3935 - sparse_categorical_accuracy: 0.8341 - val_loss: 0.5596 - val_sparse_categorical_accuracy: 0.7893
Epoch 6/50
1114/1114 [==============================] - 2s 2ms/step - loss: 0.3724 - spars

In [20]:
model1.evaluate(X_test_scaled, y_test)

verbose1 = model1.get_verbose()

comparative_results.loc['NoSelection'] = [verbose1['results']['loss'], 
                                          verbose1['results']['accuracy'], 
                                          verbose1['results']['f1'], 
                                          verbose1['selected_features'].shape[0], 
                                          verbose1['selected_features']]
comparative_results

372/372 [==============================] - 0s 940us/step - loss: 0.5351 - sparse_categorical_accuracy: 0.7822


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,0.535137,0.782155,0.769353,9214,"[amount_tsh, gps_height, longitude, latitude, ..."


* MonoFADL results

In [21]:
model2 = MonoFADLModel(
    n_inputs=X_train_scaled.columns.values.shape[0],
    n_class=y_train.unique().shape[0]
)

model2.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

model2.model.save('results/water_pump/water_pump_MonoFADLModel.h5')
with open('results/water_pump/water_pump_MonoFADLModel_history.pkl', 'wb') as f:
    pickle.dump(model2.history, f)

Epoch 1/50
1114/1114 [==============================] - 5s 4ms/step - loss: -510.5695 - sparse_categorical_accuracy: 0.5425 - val_loss: -1024.2600 - val_sparse_categorical_accuracy: 0.5431
Epoch 2/50
1114/1114 [==============================] - 3s 2ms/step - loss: -1536.8816 - sparse_categorical_accuracy: 0.5431 - val_loss: -2050.6714 - val_sparse_categorical_accuracy: 0.5431
Epoch 3/50
1114/1114 [==============================] - 3s 2ms/step - loss: -2563.3113 - sparse_categorical_accuracy: 0.5431 - val_loss: -3077.0881 - val_sparse_categorical_accuracy: 0.5431
Epoch 4/50
1114/1114 [==============================] - 3s 2ms/step - loss: -3589.6462 - sparse_categorical_accuracy: 0.5431 - val_loss: -4103.3457 - val_sparse_categorical_accuracy: 0.5431
Epoch 5/50
1114/1114 [==============================] - 3s 2ms/step - loss: -4615.9141 - sparse_categorical_accuracy: 0.5431 - val_loss: -5129.6196 - val_sparse_categorical_accuracy: 0.5431
Epoch 6/50
1114/1114 [=============================

In [22]:
model2.evaluate(X_test_scaled, y_test)

verbose2 = model2.get_verbose()

comparative_results.loc['MonoFADL'] = [verbose2['results']['loss'], 
                                          verbose2['results']['accuracy'], 
                                          verbose2['results']['f1'], 
                                          verbose2['selected_features'].shape[0], 
                                          verbose2['selected_features']]
comparative_results

372/372 [==============================] - 0s 1ms/step - loss: -51341.2148 - sparse_categorical_accuracy: 0.5431


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,0.535137,0.782155,0.769353,9214,"[amount_tsh, gps_height, longitude, latitude, ..."
MonoFADL,-51341.214844,0.543098,0.382290,0,[]


In [27]:
verbose2['predictionsproba'][:10]

array([[0.3867015 , 0.07419788, 0.53910065],
       [0.3867015 , 0.07419788, 0.53910065],
       [0.3867015 , 0.07419788, 0.53910065],
       [0.3867015 , 0.07419788, 0.53910065],
       [0.3867015 , 0.07419788, 0.53910065],
       [0.3867015 , 0.07419788, 0.53910065],
       [0.3867015 , 0.07419788, 0.53910065],
       [0.3867015 , 0.07419788, 0.53910065],
       [0.3867015 , 0.07419788, 0.53910065],
       [0.3867015 , 0.07419788, 0.53910065]], dtype=float32)

* MultiFADL One-versus-Rest results

In [23]:
model3 = MultiFADLModelOvR(
)

model3.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

for clas, model in model3.models.items():
    model.model.save(f'results/water_pump/water_pump_MultiFADLModel_{clas}.h5')    
    with open(f'results/water_pump/water_pump_MultiFADLModel_history_{clas}.pkl', 'wb') as f:
        pickle.dump(model.history, f)

--> Training model class 2 vs rest
Epoch 1/50
1114/1114 [==============================] - 9s 4ms/step - loss: -511.0155 - binary_accuracy: 0.5413 - val_loss: -1024.8103 - val_binary_accuracy: 0.5431
Epoch 2/50
1114/1114 [==============================] - 3s 3ms/step - loss: -1537.4679 - binary_accuracy: 0.5431 - val_loss: -2051.2776 - val_binary_accuracy: 0.5431
Epoch 3/50
1114/1114 [==============================] - 3s 2ms/step - loss: -2563.9282 - binary_accuracy: 0.5431 - val_loss: -3077.7151 - val_binary_accuracy: 0.5431
Epoch 4/50
1114/1114 [==============================] - 3s 2ms/step - loss: -3590.2732 - binary_accuracy: 0.5431 - val_loss: -4103.9819 - val_binary_accuracy: 0.5431
Epoch 5/50
1114/1114 [==============================] - 3s 3ms/step - loss: -4616.5376 - binary_accuracy: 0.5431 - val_loss: -5130.2505 - val_binary_accuracy: 0.5431
Epoch 6/50
1114/1114 [==============================] - 3s 3ms/step - loss: -5642.8232 - binary_accuracy: 0.5431 - val_loss: -6156.6338 

In [24]:
model3.evaluate(X_test_scaled, y_test)

verbose3 = model3.get_verbose()

comparative_results.loc['MultiFADL'] = [verbose3['results']['loss'], 
                                          verbose3['results']['accuracy'], 
                                          verbose3['results']['f1'], 
                                          {clas: verbose3['selected_features'][clas].shape[0] for clas in verbose3['selected_features']}, 
                                          verbose3['selected_features']]

comparative_results

--> Evaluating model class 2 vs rest
372/372 [==============================] - 0s 1ms/step - loss: -51341.9688 - binary_accuracy: 0.5431
{'loss': -51341.96875, 'accuracy': 0.5430976152420044, 'f1': 0.2865803089052454}
--> Evaluating model class 0 vs rest
372/372 [==============================] - 0s 1ms/step - loss: -51341.9570 - binary_accuracy: 0.6157
{'loss': -51341.95703125, 'accuracy': 0.6157407164573669, 'f1': 0.46930383105168205}
--> Evaluating model class 1 vs rest
372/372 [==============================] - 0s 1ms/step - loss: -51341.9922 - binary_accuracy: 0.9274
{'loss': -51341.9921875, 'accuracy': 0.9273568987846375, 'f1': 0.8924043318570987}


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,0.535137,0.782155,0.769353,9214,"[amount_tsh, gps_height, longitude, latitude, ..."
MonoFADL,-51341.2,0.543098,0.38229,0,[]
MultiFADL,"{2: -51341.96875, 0: -51341.95703125, 1: -5134...","{2: 0.5430976152420044, 0: 0.6157407164573669,...","{2: 0.2865803089052454, 0: 0.46930383105168205...","{2: 0, 0: 0, 1: 0, 'global': 0}","{2: [], 0: [], 1: [], 'global': []}"


In [25]:
comparative_results.to_csv('results/water_pump/water_pump_ComparativeResults.csv')

In [28]:
verbose3['results']['accuracy'], verbose3['results']['f1']

({2: 0.5430976152420044,
  0: 0.6157407164573669,
  1: 0.9273568987846375,
  'global': 0.5430976430976431},
 {2: 0.2865803089052454,
  0: 0.46930383105168205,
  1: 0.8924043318570987,
  'global': 0.38228954759611533})

In [26]:
for clase, probs in verbose3['predictionsproba'].items():
    print(f'{clase}: {probs[:10]}')

2: [[0.5391903]
 [0.5391903]
 [0.5391903]
 [0.5391903]
 [0.5391903]
 [0.5391903]
 [0.5391903]
 [0.5391903]
 [0.5391903]
 [0.5391903]]
0: [[0.38436526]
 [0.38436526]
 [0.38436526]
 [0.38436526]
 [0.38436526]
 [0.38436526]
 [0.38436526]
 [0.38436526]
 [0.38436526]
 [0.38436526]]
1: [[0.07314771]
 [0.07314771]
 [0.07314771]
 [0.07314771]
 [0.07314771]
 [0.07314771]
 [0.07314771]
 [0.07314771]
 [0.07314771]
 [0.07314771]]
global: [[0.38436526 0.07314771 0.5391903 ]
 [0.38436526 0.07314771 0.5391903 ]
 [0.38436526 0.07314771 0.5391903 ]
 [0.38436526 0.07314771 0.5391903 ]
 [0.38436526 0.07314771 0.5391903 ]
 [0.38436526 0.07314771 0.5391903 ]
 [0.38436526 0.07314771 0.5391903 ]
 [0.38436526 0.07314771 0.5391903 ]
 [0.38436526 0.07314771 0.5391903 ]
 [0.38436526 0.07314771 0.5391903 ]]
